# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2512762681284,
 'week52high': 162.81,
 'week52low': 108.28,
 'week52highSplitAdjustOnly': 161.31,
 'week52lowSplitAdjustOnly': 109.65,
 'week52change': 0.31455007570361265,
 'sharesOutstanding': 17055712510,
 'float': 0,
 'avg10Volume': 69437783,
 'avg30Volume': 77380692,
 'day200MovingAvg': 141.91,
 'day50MovingAvg': 153.11,
 'employees': 149420,
 'ttmEPS': 5.1,
 'ttmDividendRate': 0.888530591372742,
 'dividendYield': 0.005879601246631598,
 'nextDividendDate': '',
 'exDividendDate': '2021-07-24',
 'nextEarningsDate': '2021-10-17',
 'peRatio': 28.89712451140892,
 'beta': 1.4171780615288998,
 'maxChangePercent': 58.94676082291031,
 'year5ChangePercent': 4.559592615236595,
 'year2ChangePercent': 1.54188486427401,
 'year1ChangePercent': 0.3029680283493821,
 'ytdChangePercent': 0.1300447786368821,
 'month6ChangePercent': 0.1368113673886781,
 'month3ChangePercent': 0.014835848377401584,
 'month1ChangePercent': 0.01969455304401752,
 'day30ChangePerc

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.3029680283493821

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                data[symbol]['stats']['year1ChangePercent'],
                'N/A'
            ],
            index = my_columns),
            ignore_index = True
        )
final_dataframe.head()

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,163.49,0.495983,N/A
1,AAL,19.27,0.459821,N/A
2,AAP,233.75,0.545870,N/A
3,AAPL,155.78,0.294632,N/A
4,ABBV,110.63,0.362085,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,23,ALB,242.54,1.485253,N/A
1,82,CBRE,108.48,1.277123,N/A
2,29,AMAT,136.38,1.270975,N/A
3,89,CF,62.41,1.220987,N/A
4,98,CMA,90.24,1.041400,N/A
5,56,BAC,48.83,0.985512,N/A
6,18,AIG,60.72,0.945386,N/A
7,37,ANET,393.95,0.854740,N/A
8,90,CFG,52.07,0.833946,N/A
9,34,AMP,302.50,0.818196,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number! \n Please try again.')
        portfolio_size = input('Enter the size of your portfolio:')
portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe.loc[i, 'Price'])
    
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,23,ALB,242.54,1.485253,82
1,82,CBRE,108.48,1.277123,184
2,29,AMAT,136.38,1.270975,146
3,89,CF,62.41,1.220987,320
4,98,CMA,90.24,1.041400,221
5,56,BAC,48.83,0.985512,409
6,18,AIG,60.72,0.945386,329
7,37,ANET,393.95,0.854740,50
8,90,CFG,52.07,0.833946,384
9,34,AMP,302.50,0.818196,66


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy',
    'One-Year Price Return',
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [
                symbol,
                data[symbol]['price'],
                'N/A',
                data[symbol]['stats']['year1ChangePercent'],
                'N/A',
                data[symbol]['stats']['month6ChangePercent'],
                'N/A',
                data[symbol]['stats']['month3ChangePercent'],
                'N/A',
                data[symbol]['stats']['month1ChangePercent'],
                'N/A',
                'N/A'
            ],
            index = hqm_columns),
            ignore_index = True
        )
        
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.62,N/A,0.507315,N/A,0.180258,N/A,0.0531578,N/A,-0.0845461,N/A,N/A
1,AAL,19.71,N/A,0.473284,N/A,-0.0473977,N/A,-0.0961658,N/A,-0.0698496,N/A,N/A
2,AAP,241.69,N/A,0.562916,N/A,0.177243,N/A,0.112759,N/A,0.0988042,N/A,N/A
3,AAPL,155.38,N/A,0.295961,N/A,0.133239,N/A,0.0147253,N/A,0.0196356,N/A,N/A
4,ABBV,112.65,N/A,0.36651,N/A,0.0152092,N/A,-0.0613623,N/A,0.0391438,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.71,N/A,0.296485,N/A,0.0844349,N/A,0.0578711,N/A,0.00850799,N/A,N/A
501,ZBH,152.10,N/A,0.0581024,N/A,-0.154079,N/A,-0.0517847,N/A,0.034236,N/A,N/A
502,ZBRA,544.09,N/A,0.768689,N/A,0.0873922,N/A,-0.0161836,N/A,-0.0363142,N/A,N/A
503,ZION,67.87,N/A,1.14125,N/A,0.27967,N/A,0.331773,N/A,0.169774,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = [
                'One-Year',
                'Six-Month',
                'Three-Month',
                'One-Month'
                ]

for row in hqm_dataframe.index:
    for time_period in time_periods:
         if hqm_dataframe.loc[row, f'{time_period} Price Return'] == None:
            hqm_dataframe.loc[row, f'{time_period} Price Return'] = 0
            
for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.62,N/A,0.507315,0.669307,0.180258,0.714851,0.0531578,0.532673,-0.0845461,0.019802,N/A
1,AAL,19.71,N/A,0.473284,0.627723,-0.0473977,0.209901,-0.0961658,0.0831683,-0.0698496,0.0415842,N/A
2,AAP,241.69,N/A,0.562916,0.720792,0.177243,0.70297,0.112759,0.746535,0.0988042,0.780198,N/A
3,AAPL,155.38,N/A,0.295961,0.427723,0.133239,0.613861,0.0147253,0.380198,0.0196356,0.39604,N/A
4,ABBV,112.65,N/A,0.36651,0.516832,0.0152092,0.362376,-0.0613623,0.142574,0.0391438,0.524752,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.71,N/A,0.296485,0.429703,0.0844349,0.524752,0.0578711,0.552475,0.00850799,0.334653,N/A
501,ZBH,152.10,N/A,0.0581024,0.166337,-0.154079,0.0633663,-0.0517847,0.156436,0.034236,0.483168,N/A
502,ZBRA,544.09,N/A,0.768689,0.849505,0.0873922,0.536634,-0.0161836,0.257426,-0.0363142,0.108911,N/A
503,ZION,67.87,N/A,1.14125,0.944554,0.27967,0.879208,0.331773,0.974257,0.169774,0.912871,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row,'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,164.62,N/A,0.507315,0.669307,0.180258,0.714851,0.0531578,0.532673,-0.0845461,0.019802,0.484158
1,AAL,19.71,N/A,0.473284,0.627723,-0.0473977,0.209901,-0.0961658,0.0831683,-0.0698496,0.0415842,0.240594
2,AAP,241.69,N/A,0.562916,0.720792,0.177243,0.70297,0.112759,0.746535,0.0988042,0.780198,0.737624
3,AAPL,155.38,N/A,0.295961,0.427723,0.133239,0.613861,0.0147253,0.380198,0.0196356,0.39604,0.454455
4,ABBV,112.65,N/A,0.36651,0.516832,0.0152092,0.362376,-0.0613623,0.142574,0.0391438,0.524752,0.386634
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,132.71,N/A,0.296485,0.429703,0.0844349,0.524752,0.0578711,0.552475,0.00850799,0.334653,0.460396
501,ZBH,152.10,N/A,0.0581024,0.166337,-0.154079,0.0633663,-0.0517847,0.156436,0.034236,0.483168,0.217327
502,ZBRA,544.09,N/A,0.768689,0.849505,0.0873922,0.536634,-0.0161836,0.257426,-0.0363142,0.108911,0.438119
503,ZION,67.87,N/A,1.14125,0.944554,0.27967,0.879208,0.331773,0.974257,0.169774,0.912871,0.927723


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace=True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace = True, drop=True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,42.43,N/A,3.60964,1,0.965492,0.99802,0.571211,0.99802,0.328802,0.99604,0.99802
1,MRO,16.80,N/A,2.96276,0.99802,0.633498,0.990099,0.435925,0.994059,0.385368,0.99802,0.99505
2,FANG,113.51,N/A,2.87378,0.99604,0.483983,0.978218,0.41114,0.992079,0.305222,0.992079,0.989604
3,SIVB,779.25,N/A,1.6749,0.988119,0.461922,0.976238,0.332438,0.978218,0.250487,0.976238,0.979703
4,COP,78.30,N/A,1.47793,0.978218,0.581286,0.986139,0.347925,0.980198,0.218364,0.964356,0.977228
5,PXD,196.15,N/A,1.37743,0.974257,0.334839,0.942574,0.351124,0.982178,0.289204,0.990099,0.972277
6,OXY,34.39,N/A,2.42029,0.994059,0.415695,0.972277,0.261038,0.932673,0.277989,0.988119,0.971782
7,EOG,95.00,N/A,1.54938,0.982178,0.376738,0.960396,0.26116,0.934653,0.245565,0.970297,0.961881
8,MOS,42.24,N/A,1.31879,0.968317,0.286385,0.89505,0.4066,0.988119,0.276355,0.986139,0.959406
9,HAL,26.43,N/A,0.999047,0.916832,0.36127,0.954455,0.274838,0.948515,0.313207,0.994059,0.953465


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio:100000


In [15]:
position_size = float(portfolio_size) / len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i, 'Price'])
    
hqm_dataframe

C:\Users\jhtc9\OneDrive\Desktop\algorithmic-trading-python\env\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,DVN,42.43,47,3.60964,1,0.965492,0.99802,0.571211,0.99802,0.328802,0.99604,0.99802
1,MRO,16.80,119,2.96276,0.99802,0.633498,0.990099,0.435925,0.994059,0.385368,0.99802,0.99505
2,FANG,113.51,17,2.87378,0.99604,0.483983,0.978218,0.41114,0.992079,0.305222,0.992079,0.989604
3,SIVB,779.25,2,1.6749,0.988119,0.461922,0.976238,0.332438,0.978218,0.250487,0.976238,0.979703
4,COP,78.30,25,1.47793,0.978218,0.581286,0.986139,0.347925,0.980198,0.218364,0.964356,0.977228
5,PXD,196.15,10,1.37743,0.974257,0.334839,0.942574,0.351124,0.982178,0.289204,0.990099,0.972277
6,OXY,34.39,58,2.42029,0.994059,0.415695,0.972277,0.261038,0.932673,0.277989,0.988119,0.971782
7,EOG,95.00,21,1.54938,0.982178,0.376738,0.960396,0.26116,0.934653,0.245565,0.970297,0.961881
8,MOS,42.24,47,1.31879,0.968317,0.286385,0.89505,0.4066,0.988119,0.276355,0.986139,0.959406
9,HAL,26.43,75,0.999047,0.916832,0.36127,0.954455,0.274838,0.948515,0.313207,0.994059,0.953465


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {
    'A': ['Ticker', string_template],
    'B': ['Price', dollar_template],
    'C': ['Number of Shares to Buy', integer_template],
    'D': ['One-Year Price Return', percent_template],
    'E': ['One-Year Return Percentile', percent_template],
    'F': ['Six-Month Price Return', percent_template],
    'G': ['Six-Month Return Percentile', percent_template],
    'H': ['Three-Month Price Return', percent_template],
    'I': ['Three-Month Return Percentile', percent_template],
    'J': ['One-Month Price Return', percent_template],
    'K': ['One-Month Return Percentile', percent_template],
    'L': ['HQM Score', percent_template]
}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}' , 25 , column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()